In [1]:
import sys
import math
import logging
import warnings
import numpy as np
import pandas as pd
import tensorflow_text
import tensorflow as tf
import tensorflow_hub as hub
from scipy.stats import pearsonr

logging.disable(logging.WARNING)
warnings.filterwarnings('ignore')
np.set_printoptions(threshold=sys.maxsize)

2024-05-08 17:25:15.027218: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-08 17:25:15.068613: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-08 17:25:15.762154: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Load all the data
trainpath = 'data/train-en-es.csv'
testpath = 'data/test-en-es.csv'
valpath = 'data/validation-en-es.csv'

traindata = pd.read_csv(trainpath)
testdata = pd.read_csv(testpath)
valdata = pd.read_csv(valpath)

traindata['score'] = traindata['similarity_score'].apply(lambda x: (x)/5.0)
testdata['score'] = testdata['similarity_score'].apply(lambda x: (x)/5.0)
valdata['score'] = valdata['similarity_score'].apply(lambda x: (x)/5.0)

In [3]:
modelpath = "https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/3"
use = hub.load(modelpath)

def pearson_corr(y_true, y_pred):
    corr, _ = pearsonr(y_true, y_pred)
    return corr

def embed(input):
    return use(input)

def run_sts_benchmark(batch):
  sts_encode1 = tf.nn.l2_normalize(embed(tf.constant(batch['sentence1'].tolist())), axis=1)
  sts_encode2 = tf.nn.l2_normalize(embed(tf.constant(batch['sentence2'].tolist())), axis=1)
  cosine_similarities = tf.reduce_sum(tf.multiply(sts_encode1, sts_encode2), axis=1)
  clip_cosine_similarities = tf.clip_by_value(cosine_similarities, -1.0, 1.0)
  scores = 1.0 - tf.acos(clip_cosine_similarities) / math.pi
  return scores

2024-05-08 17:25:17.225780: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-08 17:25:17.226347: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
scores = []
for batch in np.array_split(traindata, 32):
  scores.extend(run_sts_benchmark(batch))

corr = pearson_corr(traindata['score'], scores)
print("Pearson correlation coefficient: {:.2f}".format(corr))

Pearson correlation coefficient: -0.01


In [5]:
scores = []
for batch in np.array_split(testdata, 32):
  scores.extend(run_sts_benchmark(batch))

corr = pearson_corr(testdata['score'], scores)
print("Pearson correlation coefficient: {:.2f}".format(corr))

Pearson correlation coefficient: 0.04


In [6]:
scores = []
for batch in np.array_split(valdata, 32):
  scores.extend(run_sts_benchmark(batch))

corr = pearson_corr(valdata['score'], scores)
print("Pearson correlation coefficient: {:.2f}".format(corr))

Pearson correlation coefficient: 0.01


In [7]:
class USELayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(USELayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.use = hub.load(modelpath)
        super(USELayer, self).build(input_shape)

    def call(self, inputs):
        return self.use(inputs)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], 512)

input1 = tf.keras.layers.Input(shape=[], dtype=tf.string)
input2 = tf.keras.layers.Input(shape=[], dtype=tf.string)

embedding1 = USELayer()(input1)
embedding2 = USELayer()(input2)

concatenated = tf.keras.layers.Concatenate()([embedding1, embedding2])
dense1 = tf.keras.layers.Dense(512, activation='relu')(concatenated)
output = tf.keras.layers.Dense(1, activation='linear')(dense1)

model = tf.keras.Model(inputs=[input1, input2], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

x_train = [traindata['sentence1'], traindata['sentence2']]
y_train = np.array(traindata['score'])
model.fit(x_train, y_train, epochs=10, validation_split=0.2, batch_size=32)

Epoch 1/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 176s 1s/step - loss: 0.0938 - mse: 0.0938 - val_loss: 0.0977 - val_mse: 0.0977
Epoch 2/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - loss: 0.0720 - mse: 0.0720 - val_loss: 0.0951 - val_mse: 0.0951
Epoch 3/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 163s 1s/step - loss: 0.0699 - mse: 0.0699 - val_loss: 0.0982 - val_mse: 0.0982
Epoch 4/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 170s 1s/step - loss: 0.0665 - mse: 0.0665 - val_loss: 0.0964 - val_mse: 0.0964
Epoch 5/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 191s 1s/step - loss: 0.0635 - mse: 0.0635 - val_loss: 0.1154 - val_mse: 0.1154
Epoch 6/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 203s 1s/step - loss: 0.0606 - mse: 0.0606 - val_loss: 0.1000 - val_mse: 0.1000
Epoch 7/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 261s 1s/step - loss: 0.0544 - mse: 0.0544 - val_loss: 0.1016 - val_mse: 0.1016
Epoch 8/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 220s 2s/step - loss: 0.0476 - mse: 0.0476 - val_loss: 0.1041 - val_mse: 0.1041
Epoch 9/10
144/144 ━━━━━━━━━━━━━━━━━━━━ 185s 1s/

In [8]:
y_pred = model.predict(x_train).flatten()
corr = pearsonr(y_train, y_pred)
print("Pearson correlation coefficient:", corr.statistic)

180/180 ━━━━━━━━━━━━━━━━━━━━ 177s 952ms/step
Pearson correlation coefficient: 0.6725686443548434


In [9]:
x_val = [valdata['sentence1'], valdata['sentence2']]
y_val = np.array(valdata['score'])

y_pred = model.predict(x_val).flatten()
corr = pearsonr(y_val, y_pred)
print("Pearson correlation coefficient:", corr.statistic)

47/47 ━━━━━━━━━━━━━━━━━━━━ 45s 968ms/step
Pearson correlation coefficient: 0.11662998527877845


In [10]:
x_test = [testdata['sentence1'], testdata['sentence2']]
y_test = np.array(testdata['score'])

y_pred = model.predict(x_test).flatten()
corr = pearsonr(y_test, y_pred)
print("Pearson correlation coefficient:", corr.statistic)

44/44 ━━━━━━━━━━━━━━━━━━━━ 37s 845ms/step
Pearson correlation coefficient: 0.19142071585823445
